In [6]:
from datetime import datetime, timedelta
import json
from collections import deque

class DailyMission:
    """
    Daily Mission class with
        name (str): Name of the mission
        description (str): Detailed description of the mission
        xp_reward (int): Experience points awarded for completion
        category (str): Mission category (strength, stamina, willpower)
        required_level (int): Minimum player level needed to unlock
        completed (bool): Track whether mission is completed
        children (list): Sub-missions for hierarchical structure
    """
    def __init__(self, name, description, xp_reward, category, required_level=1, children=None):
        self.name = name
        self.description = description
        self.xp_reward = xp_reward
        self.category = category
        self.required_level = required_level
        self.completed = False
        self.children = children or []

class Player:
    """Main player class with
        name (str): Player name
        level (int): Current player level
        last_active (date): Last play date
        daily_missions (list): Today's available missions
        pending_missions_queue (deque): Missions waiting to be completed
        total_xp (dict): XP accumulated per category
        levels (dict): Current level per category
        mission_tree (DailyMission): Root of mission hierarchy
    """
    def __init__(self, name):
        self.name = name
        self.level = 1
        self.last_active = datetime.now().date() - timedelta(days=1)
        self.daily_missions = []
        self.pending_missions_queue = deque()
        self.total_xp = {"strength": 0, "stamina": 0, "willpower": 0}
        self.levels = {"strength": 1, "stamina": 1, "willpower": 1}

        # Mission tree structure
        self.mission_tree = DailyMission("Root", "All Missions", 0, "root", 0, [
            DailyMission("Beginner", "Beginner Missions", 0, "root", 1, [
                DailyMission("Run", "Run 3 km", 10, "stamina"),
                DailyMission("Push Up", "10 push ups", 10, "strength"),
                DailyMission("Sit Up", "10 sit ups", 10, "strength"),
                DailyMission("Eat Less", "Eat 100 calories less", 10, "willpower"),
                DailyMission("Protein", "Eat 50g protein", 10, "willpower")
            ]),
            DailyMission("Intermediate", "Intermediate Missions", 0, "root", 2, [
                DailyMission("Run", "Run 5 km", 15, "stamina"),
                DailyMission("Push Up", "15 push ups", 15, "strength"),
                DailyMission("Sit Up", "15 sit ups", 15, "strength"),
                DailyMission("Eat Less", "Eat 150 calories less", 15, "willpower"),
                DailyMission("Protein", "Eat 75g protein", 15, "willpower")
            ]),
            DailyMission("Advanced", "Advanced Missions", 0, "root", 3, [
                DailyMission("Run", "Run 7 km", 20, "stamina"),
                DailyMission("Push Up", "20 push ups", 20, "strength"),
                DailyMission("Sit Up", "20 sit ups", 20, "strength"),
                DailyMission("Eat Less", "Eat 200 calories less", 20, "willpower"),
                DailyMission("Protein", "Eat 100g protein", 20, "willpower")
            ])
        ])

        self.generate_daily_missions()

    def generate_daily_missions(self):
        #Generate the default set of 5 daily missions
        self.daily_missions = [
            DailyMission("Run", "Run 3 km", 10, "stamina"),
            DailyMission("Push Up", "10 push ups", 10, "strength"),
            DailyMission("Sit Up", "10 sit ups", 10, "strength"),
            DailyMission("Eat Less", "Eat 100 calories less", 10, "willpower"),
            DailyMission("Protein", "Eat 50g protein", 10, "willpower")
        ]

        self.pending_missions_queue = deque([m for m in self.daily_missions if not m.completed])

    def refresh_day(self):
        #Reset daily missions and update last active date.
        self.last_active = datetime.now().date()
        self.generate_daily_missions()
        self.save_progress()

    def complete_mission(self, mission):
        #Mark a mission as completed and update player stats
        if mission.completed:
            return False

        mission.completed = True
        self.total_xp[mission.category] += mission.xp_reward
        if self.pending_missions_queue and self.pending_missions_queue[0] == mission:
            self.pending_missions_queue.popleft()

        print(f"\n Mission complete! +{mission.xp_reward} XP to {mission.category}")

        # Check for level ups
        for cat in self.total_xp:
            if self.total_xp[cat] >= self.levels[cat] * 100:
                self.levels[cat] += 1
                print(f"\n {cat.capitalize()} level up! Now level {self.levels[cat]}")

        # Update player level based on average of all categories
        avg_level = sum(self.levels.values()) // 3
        if avg_level > self.level:
            self.level = avg_level
            print(f"\n Player level increased to {self.level}! New missions unlocked!")

        self.save_progress()
        self.show_missions()
        return True

    def show_missions(self):
        #Display current missions and player stats
        print(f"\n=== DAILY MISSIONS (Level {self.level}) ===")
        print(f"Player: {self.name}")
        print(f"Levels: Strength {self.levels['strength']} | Stamina {self.levels['stamina']} | Willpower {self.levels['willpower']}")
        print(f"XP: Strength {self.total_xp['strength']} | Stamina {self.total_xp['stamina']} | Willpower {self.total_xp['willpower']}\n")

        if not self.daily_missions:
            print("No missions available. Generating new missions...")
            self.generate_daily_missions()

        for i, mission in enumerate(self.daily_missions, 1):
            status = "✓" if mission.completed else " "
            print(f"{i}. [{status}] {mission.name}: {mission.description} ({mission.xp_reward} XP)")

    def show_mission_tree(self):
        #Display the complete mission hierarchy with unlock status
        print("\n=== MISSION TREE ===")
        print("(Unlocked missions based on your level)")

        def print_tree(node, indent=0):
            prefix = "  " * indent
            if node.required_level > self.level:
                print(f"{prefix}- {node.name} [Locked - Level {node.required_level} required]")
            else:
                if node.children:
                    print(f"{prefix}+ {node.name}")
                else:
                    status = "✓" if any(m.name == node.name and m.completed for m in self.daily_missions) else " "
                    print(f"{prefix}- [{status}] {node.name}")

                for child in node.children:
                    print_tree(child, indent + 1)

        print_tree(self.mission_tree)
        print("\nNote: Complete missions to level up and unlock new challenges!")

    def save_progress(self, filename="progress.json"):
        #Save player progress to file
        data = {
            "name": self.name,
            "level": self.level,
            "total_xp": self.total_xp,
            "levels": self.levels,
            "last_active": self.last_active.isoformat()
        }
        with open(filename, "w") as f:
            json.dump(data, f)

    @classmethod
    def load_progress(cls, filename="progress.json"):
        """Load player progress from file"""
        try:
            with open(filename, "r") as f:
                data = json.load(f)
            player = cls(data["name"])
            player.level = data.get("level", 1)
            player.total_xp = data.get("total_xp", {"strength": 0, "stamina": 0, "willpower": 0})
            player.levels = data.get("levels", {"strength": 1, "stamina": 1, "willpower": 1})
            player.last_active = datetime.fromisoformat(data["last_active"]).date()
            player.generate_daily_missions()
            return player
        except FileNotFoundError:
            return None

def main():
    #Let's start the game
    print("Notification!")
    print("\n=== Welcome to Soloing Leveling System ===")
    print("You have acquired the qualifications to be a Player.")
    choice = input("Will you accept? (Y/N): ").strip().upper()

    if choice != 'Y':
        print("The quest awaits when you're ready...")
        return

    player = Player.load_progress()
    if player:
        print(f"\nWelcome back, {player.name}!")
    else:
        player_name = "Player"
        player = Player(player_name)
        player.save_progress()
        print("\n[You've become a Player]")
        print("\n[Daily Quest has arrived]")

    while True:
        player.show_missions()
        print("\nOptions:")
        print("1. Complete next mission")
        print("2. Start new day (refresh missions)")
        print("3. View mission tree")
        print("4. Exit")

        choice = input("\nSelect an option (1-4): ").strip()

        if choice == "1":
            if not player.pending_missions_queue:
                print("\n All missions completed for today! Great job!")
                continue

            next_mission = player.pending_missions_queue[0]
            player.complete_mission(next_mission)

            if not player.pending_missions_queue:
                print("\n All missions completed for today! Great job!")

        elif choice == "2":
            player.refresh_day()
            print("\n New day! New missions awaits! \n")

        elif choice == "3":
            player.show_mission_tree()
            input("\nPress Enter to continue...")

        elif choice == "4":
            print("\nGoodbye! Keep up the good work!")
            break

        else:
            print("\nInvalid choice. Please select 1, 2, 3, or 4")

        input("\nPress Enter to continue...")

if __name__ == "__main__":
    main()

Notification!

=== Welcome to Soloing Leveling System ===
You have acquired the qualifications to be a Player.
Will you accept? (Y/N): y

Welcome back, Player!

=== DAILY MISSIONS (Level 1) ===
Player: Player
Levels: Strength 1 | Stamina 1 | Willpower 1
XP: Strength 40 | Stamina 20 | Willpower 40

1. [ ] Run: Run 3 km (10 XP)
2. [ ] Push Up: 10 push ups (10 XP)
3. [ ] Sit Up: 10 sit ups (10 XP)
4. [ ] Eat Less: Eat 100 calories less (10 XP)
5. [ ] Protein: Eat 50g protein (10 XP)

Options:
1. Complete next mission
2. Start new day (refresh missions)
3. View mission tree
4. Exit

Select an option (1-4): 1

 Mission complete! +10 XP to stamina

=== DAILY MISSIONS (Level 1) ===
Player: Player
Levels: Strength 1 | Stamina 1 | Willpower 1
XP: Strength 40 | Stamina 30 | Willpower 40

1. [✓] Run: Run 3 km (10 XP)
2. [ ] Push Up: 10 push ups (10 XP)
3. [ ] Sit Up: 10 sit ups (10 XP)
4. [ ] Eat Less: Eat 100 calories less (10 XP)
5. [ ] Protein: Eat 50g protein (10 XP)

Press Enter to continue.